# In this Notebook we look at a framework to store the data so that is simple to gather and experiment with different features

In [1]:
# Modues to be used
import ast 
import pickle

import pandas as pd
import numpy as np

from datetime import datetime
from re import sub

## Read CSV Files

In [2]:
# Read the data
MASTER_CSV_PATH = '~/Desktop/csgo-csv/'

# Years to be considered
year_vec = ['2012','2013','2014','2015','2016','2017','2018','2019']
#year_vec = ['2019']
#year_vec = ['2018', '2019']

# Empty data frames
df_player = pd.DataFrame( { } )
df_team   = pd.DataFrame( { } )

df_event = pd.read_csv(MASTER_CSV_PATH+'events.csv')
df_event = df_event.set_index('id')

# Append the data from each year
for year in year_vec:
        
    file_name = MASTER_CSV_PATH + year + '-player-lvl.csv'
    df_player = df_player.append( pd.read_csv( file_name ), ignore_index=True, sort=False )
    
    file_name = MASTER_CSV_PATH + year + '-team-lvl.csv'
    df_team   = df_team.append( pd.read_csv( file_name ), ignore_index=True, sort=False )

## Create a player dictionary with entries that have all the matches the player has participated on

In [3]:
# Now we create a dictionary for all the maps
dict_map = {}

# Initialize
for map_id in df_team.map_id.unique():
    dict_map[map_id] = {}

# We extract the values to a vector so that it's faster
map_id_vec  = df_team.map_id.values
score_vec   = df_team.team_score.values
team_id_vec = df_team.team_id.values

for ind in range( 0, len( map_id_vec ) ):

    map_id  = map_id_vec[ind]
    team_id = team_id_vec[ind]

    df_tmp = df_player[ df_player['map_id']==map_id ]

    dict_map[map_id][team_id]               = {}
    dict_map[map_id][team_id]['score']      = score_vec[ind]
    dict_map[map_id][team_id]['players_id'] = df_tmp[ (df_tmp['team_id']==team_id) ]['player_id'].values

In [4]:
for map_id in dict_map:

    ind = 0
    for team_id in dict_map[map_id]:
        if ind == 0:
            team_id_1 = team_id
            score_team_1 = dict_map[map_id][team_id]['score']
        else:
            team_id_2 = team_id
            score_team_2 = dict_map[map_id][team_id]['score']
        ind = ind+1

    if score_team_1 > score_team_2:
        dict_map[map_id][team_id_1]['win'] = 1
        dict_map[map_id][team_id_2]['win'] = 0
    elif score_team_1 < score_team_2:
        dict_map[map_id][team_id_1]['win'] = 0
        dict_map[map_id][team_id_2]['win'] = 1
    else:
        dict_map[map_id][team_id_1]['win'] = 0
        dict_map[map_id][team_id_2]['win'] = 0

In [5]:
# Rank during events
dict_event = {}

map_to_event = {}

for event_id in df_team['event_id'].unique()[:]:

    s   = df_event.loc[event_id]['teams']
    l   = len(s)
    dic = ast.literal_eval(s[1:l-1])

    df_tmp = df_team[ df_team['event_id']==event_id ]

    for map_id in df_tmp['map_id'].unique():
        map_to_event[map_id] = event_id        

    dict_event[event_id] = {}

    for team in dic:        
        team_id = team['id']
        try:
            rank = team['rankDuringEvent']            
        except:
            rank = 0

        dict_event[event_id][team_id]         = {}
        dict_event[event_id][team_id]['rank'] = rank

# We create a more compact data frame for the team that has all the relevant information

In [6]:
dict_tmp = {}

for map_id in df_team.map_id.unique()[:]:

    df_tmp = df_team[ df_team['map_id']==map_id ] 

    dict_tmp[map_id] = {}

    dict_tmp[map_id]['map_id']     = df_tmp['map_id'].values[0]
    dict_tmp[map_id]['map']        = df_tmp['map'].values[0]
    dict_tmp[map_id]['date']       = datetime.utcfromtimestamp( df_tmp['date_unix'].values[0] / 1000 )
    dict_tmp[map_id]['event_id']   = df_tmp['event_id'].values[0]
    dict_tmp[map_id]['event_name'] = df_tmp['event_name'].values[0]

    # Info for team 1
    dict_tmp[map_id]['team_id_1']    = df_tmp['team_id'].values[0]
    dict_tmp[map_id]['team_name_1']  = df_tmp['team_name'].values[0]
    dict_tmp[map_id]['team_score_1'] = int( df_tmp['team_score'].values[0] )
    try:        
        dict_tmp[map_id]['team_kills_1']   = int( df_tmp['team_kills'].values[0] )
        dict_tmp[map_id]['team_deaths_1']  = int( df_tmp['team_deaths'].values[0] )
        dict_tmp[map_id]['team_assists_1'] = int( df_tmp['team_assists'].values[0] )
    except:
        dict_tmp[map_id]['team_kills_1']   = np.NaN
        dict_tmp[map_id]['team_deaths_1']  = np.NaN
        dict_tmp[map_id]['team_assists_1'] = np.NaN
    try:
        dict_tmp[map_id]['team_rank_1'] = dict_event[dict_tmp[map_id]['event_id']][dict_tmp[map_id]['team_id_1']]['rank']
    except:
        dict_tmp[map_id]['team_rank_1'] = 0


    # Info for team 1
    dict_tmp[map_id]['team_id_2']    = df_tmp['team_id'].values[1]
    dict_tmp[map_id]['team_name_2']  = df_tmp['team_name'].values[1]
    dict_tmp[map_id]['team_score_2'] = int( df_tmp['team_score'].values[1] )
    try:        
        dict_tmp[map_id]['team_kills_2']   = int( df_tmp['team_kills'].values[1] )
        dict_tmp[map_id]['team_deaths_2']  = int( df_tmp['team_deaths'].values[1] )
        dict_tmp[map_id]['team_assists_2'] = int( df_tmp['team_assists'].values[1] )
    except:
        dict_tmp[map_id]['team_kills_2']   = np.NaN
        dict_tmp[map_id]['team_deaths_2']  = np.NaN
        dict_tmp[map_id]['team_assists_2'] = np.NaN
    try:
        dict_tmp[map_id]['team_rank_2'] = dict_event[dict_tmp[map_id]['event_id']][dict_tmp[map_id]['team_id_2']]['rank']
    except:
        dict_tmp[map_id]['team_rank_2'] = 0

    # Winner 
    if dict_tmp[map_id]['team_score_1'] > dict_tmp[map_id]['team_score_2']:
        dict_tmp[map_id]['winner'] = 1
    elif dict_tmp[map_id]['team_score_1'] < dict_tmp[map_id]['team_score_2']:
        dict_tmp[map_id]['winner'] = 2
    else:
        dict_tmp[map_id]['winner'] = 0


df = pd.DataFrame.from_dict(dict_tmp,orient='index')

In [7]:
dict_tmp
df[:100]

,map_id,map,date,event_id,event_name,team_id_1,team_name_1,team_score_1,team_kills_1,team_deaths_1,team_assists_1,team_rank_1,team_id_2,team_name_2,team_score_2,team_kills_2,team_deaths_2,team_assists_2,team_rank_2,winner
12838,12838,NaN,2012-09-13 19:00:00,965,Go4CS:GO Cup #6,4411,NiP,16,91.0,46.0,0.0,0,4443,PRiME,3,45.0,91.0,0.0,0,1
12839,12839,NaN,2012-09-13 19:00:00,965,Go4CS:GO Cup #6,4411,NiP,16,83.0,34.0,0.0,0,4443,PRiME,1,34.0,83.0,0.0,0,1
12840,12840,NaN,2012-09-13 19:00:00,966,CyberGamer Qualifier Cup #1 by Tt eSPORTS,4444,Blight,16,81.0,37.0,0.0,0,4445,FMESPORTS,2,37.0,81.0,0.0,0,1
12841,12841,NaN,2012-09-13 19:00:00,966,CyberGamer Qualifier Cup #1 by Tt eSPORTS,4444,Blight,16,97.0,58.0,0.0,0,4445,FMESPORTS,7,57.0,97.0,0.0,0,1
12848,12848,NaN,2012-09-23 12:00:00,973,DreamHack Valencia 2012,4411,NiP,16,102.0,86.0,0.0,0,4467,VeryGames,10,86.0,102.0,0.0,0,1
12849,12849,NaN,2012-09-23 12:00:00,973,DreamHack Valencia 2012,4411,NiP,16,108.0,89.0,0.0,0,4467,VeryGames,12,89.0,108.0,0.0,0,1
12853,12853,NaN,2012-09-22 08:50:00,973,DreamHack Valencia 2012,4467,VeryGames,16,80.0,26.0,0.0,0,4473,DN-Gaming,0,26.0,80.0,0.0,0,1
12854,12854,NaN,2012-09-22 10:00:00,973,DreamHack Valencia 2012,4469,k1ck,16,92.0,81.0,0.0,0,4472,ASES,9,81.0,92.0,0.0,0,1
12855,12855,NaN,2012-09-22 11:30:00,973,DreamHack Valencia 2012,4472,ASES,16,107.0,95.0,0.0,0,4473,DN-Gaming,13,95.0,108.0,0.0,0,1
12856,12856,NaN,2012-09-22 12:30:00,973,DreamHack Valencia 2012,4467,VeryGames,16,92.0,42.0,0.0,0,4469,k1ck,3,42.0,92.0,0.0,0,1


In [8]:
dict_player = {}

#count = 0
for player_id in df_player.player_id.unique()[:]:

    dict_player[player_id] = df_player[ df_player['player_id']==player_id ].sort_values(by='date',ascending=True).reset_index()

    # Convert the dates to a date format
    team_score = []
    opponent_score = []
    date_tmp = []
    win_tmp = []
    rank_tmp = []
    prize_tmp = []

    team_id_vec = dict_player[player_id]['team_id'].values
    map_id_vec  = dict_player[player_id]['map_id'].values
    date_vec    = dict_player[player_id]['date'].values    

    for ind in range(0,len(dict_player[player_id]['date'].values[:])):        

        team_id = team_id_vec[ind]
        map_id  = map_id_vec[ind]
        event_id = map_to_event[map_id]

        for id_aux in dict_map[map_id]:
            if id_aux == team_id:
                score = dict_map[map_id][id_aux]['score']
            else:
                score_op = dict_map[map_id][id_aux]['score']

        win_flag = dict_map[map_id][team_id]['win']     

        date_tmp.append( datetime.utcfromtimestamp( date_vec[ind] / 1000 ) )
        team_score.append( score )
        opponent_score.append( score_op )
        win_tmp.append( win_flag )
        prize_tmp.append( float(0.0) )
        
        try:
            rank_tmp.append( dict_event[event_id][team_id]['rank'] )
        except:
            rank_tmp.append( 0 )
        

    dict_player[player_id]['date']       = date_tmp
    dict_player[player_id]['team_score'] = team_score
    dict_player[player_id]['op_score']   = opponent_score
    dict_player[player_id]['win']        = win_tmp
    dict_player[player_id]['team_rank']  = rank_tmp
    dict_player[player_id]['prize']      = prize_tmp
    
    dict_player[player_id] = dict_player[player_id].drop('index',axis=1).set_index('map_id')

In [9]:
# dict_player[11264][:50]
# dict_player[14557].loc[83348]

In [10]:
# Prize money 
for event_id in df_team['event_id'].unique()[:]:

    df_tmp = df_team[ df_team['event_id']==event_id ].sort_values('date_unix',ascending=False)
    try:
        # The prize distribution is a string that we convert to a dictionary
        s   = df_event.loc[event_id]['prizeDistribution']
        l   = len(s)
        dic = ast.literal_eval(s[1:l-1])        

        # date_start = datetime.utcfromtimestamp( df_event.loc[event_id]['dateStart'] / 1000 )
        # date_end   = datetime.utcfromtimestamp( df_event.loc[event_id]['dateEnd'] / 1000 )

        # Loop through each of the places of the tournament
        for place in dic:
            try:
                prize   = float(sub(r'[^\d.]', '', place['prize']))
                team_id = place['team']['id']
                map_id  = df_tmp[ (df_tmp['team_id']==team_id) ]['map_id'].unique()[0]

                # Assuming the prize gets divided evenly between players and coach 
                for player_id in dict_map[map_id][team_id]['players_id']:
                    dict_player[player_id].at[map_id,'prize'] = prize / 6.0

            except:
                continue 

    except:
        continue 

In [11]:
# dict_player[7592]

# Save Pickle files

In [12]:
with open('data/dict_map.pickle','wb') as f:
    pickle.dump(dict_map, f)

with open('data/map_to_event.pickle','wb') as f:
    pickle.dump(map_to_event, f)

with open('data/dict_event.pickle','wb') as f:
    pickle.dump(dict_event, f)

with open('data/df_map.pickle','wb') as f:
    pickle.dump(df, f)

with open('data/dict_player.pickle','wb') as f:
    pickle.dump(dict_player, f)